In [1]:
from google.colab import files
uploaded = files.upload()
!unzip -q upload_zip.zip
# dowanload the dogs_and_cats dataset
!wget https://nextcloud.rickdgray.com/index.php/s/n2ZX7n2er4z3PpP/download/dogs_and_cats.zip

!unzip -q dogs_and_cats.zip -d data

Saving upload_zip.zip to upload_zip.zip
--2024-02-20 04:46:26--  https://nextcloud.rickdgray.com/index.php/s/n2ZX7n2er4z3PpP/download/dogs_and_cats.zip
Resolving nextcloud.rickdgray.com (nextcloud.rickdgray.com)... 45.24.51.109
Connecting to nextcloud.rickdgray.com (nextcloud.rickdgray.com)|45.24.51.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 882765995 (842M) [application/zip]
Saving to: ‘dogs_and_cats.zip’

dogs_and_cats.zip   100%[===================>] 841.87M  9.89MB/s    in 79s     

2024-02-20 04:47:46 (10.7 MB/s) - ‘dogs_and_cats.zip’ saved [882765995/882765995]



In [2]:
%pylab inline
import torch
import sys
sys.path.append('..')
sys.path.append('../..')
from data import load
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device = ', device)

Populating the interactive namespace from numpy and matplotlib
device =  cuda


In [4]:
class ConvNet(torch.nn.Module):
    class Block(torch.nn.Module):
        def __init__(self, n_input, n_output, stride=1):
            super().__init__()
            self.net = torch.nn.Sequential(
              torch.nn.Conv2d(n_input, n_output, kernel_size=3, padding=1, stride=stride, bias=False),
              torch.nn.BatchNorm2d(n_output), # still using batch normalization
              torch.nn.ReLU(),
              torch.nn.Conv2d(n_output, n_output, kernel_size=3, padding=1, bias=False),
              torch.nn.BatchNorm2d(n_output),
              torch.nn.ReLU()
            )
            self.downsample = None # downsampling layer ensures the shortcut connection has same channels and spatial resolution as rest of network
            if stride != 1 or n_input != n_output: # if stride is not 1 or input/output number do not match... need to add a downsampling layer
                self.downsample = torch.nn.Sequential(torch.nn.Conv2d(n_input, n_output, 1), # create downsampling layer, add batch normalization
                                                      torch.nn.BatchNorm2d(n_output))

        def forward(self, x):
            identity = x # adding network itself
            if self.downsample is not None:
                identity = self.downsample(x) # if downsampling layer was needed, add that to output/network
            return self.net(x) + identity # adding network

    def __init__(self, layers=[32,64,128], n_input_channels=3):
        super().__init__()
        L = [torch.nn.Conv2d(n_input_channels, 32, kernel_size=7, padding=3, stride=2, bias=False),
             torch.nn.BatchNorm2d(32),
             torch.nn.ReLU(),
             torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
            ]
        c = 32
        for l in layers:
            L.append(self.Block(c, l, stride=2))
            c = l
        self.network = torch.nn.Sequential(*L)
        self.classifier = torch.nn.Linear(c, 1)

    def forward(self, x):
        # Compute the features
        z = self.network(x)
        # Global average pooling
        z = z.mean(dim=[2,3])
        # Classify
        return self.classifier(z)[:,0]

net = ConvNet()
net.train()
print( net.training )
net.eval()
print( net.training )

True
False
